<a href="https://colab.research.google.com/github/hayakzan/MLP-notations/blob/main/MLP_notations_SYNTHESIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Synthesize the results into MIDI and .wav

In [2]:
!pip install mido
!pip install midi2audio
!apt install fluidsynth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# -*- coding: utf-8 -*-

import json
from google.colab import drive
import shutil
import mido
from IPython.display import Audio


In [4]:
# mount Google Drive
drive.mount('/content/drive')

source_file_path = '/content/drive/MyDrive/pitch_rhythm.json'

destination_directory = '/content'

shutil.copy(source_file_path, destination_directory)


Mounted at /content/drive


'/content/pitch_rhythm.json'

#### Process the JSON file

In [5]:
with open('pitch_rhythm.json') as json_file:
    data = json.load(json_file)

notation_excerpt = []

for group in data:
    input_pairs = []
    for pair in group:
        input_pairs.append(pair)

    notation_excerpt.append(input_pairs)

notation_excerpt[2]


[[7.5, 0.4426416321209812], [3.5, 1.2791865053616016], [3.5, 1.66663148003785]]

In [6]:
# dictionary for notational position -> MIDI
pitch_dict = {0.0:52, 0.5:53, 1.0:55, 1.5:57, 2.0:59, 2.5:60, 3.0:62, 3.5:64, 4.0:65, 4.5:67, 5.0:69, 5.5:71, 6.0:72, 6.5:74, 7.0:76, 7.5:77, 8.0:79, 8.5:81, 9.0:83, 9.5:84, 10.0:86, 10.5:88, 11.0:89, 11.5:91, 12.0:93}


In [7]:
# get pitch and rhythm data for MIDI dict
MIDI_pitch_data = []
MIDI_rhythm_data = []
MIDI_pitches = []
MIDI_rhythms = []

for pairs in notation_excerpt[2]:
  MIDI_pitch_data.append(pairs[0])

for pairs in notation_excerpt[2]:
  MIDI_rhythm_data.append(pairs[1])

print("excerpt: ", notation_excerpt[2])
print("MIDI pitches: ", MIDI_pitch_data)
print("MIDI rhythmis: ", MIDI_rhythm_data)

excerpt:  [[7.5, 0.4426416321209812], [3.5, 1.2791865053616016], [3.5, 1.66663148003785]]
MIDI pitches:  [7.5, 3.5, 3.5]
MIDI rhythmis:  [0.4426416321209812, 1.2791865053616016, 1.66663148003785]


In [8]:
# get MIDI data
MIDI_pitches = [pitch_dict[i] for i in MIDI_pitch_data] 

for i in range(len(MIDI_rhythm_data)):
  MIDI_rhythms.append(int(240*(MIDI_rhythm_data[i]/1.235)))

print("MIDI pitches: ", MIDI_pitches)
print("MIDI rhythms: ", MIDI_rhythms)

MIDI pitches:  [77, 64, 64]
MIDI rhythms:  [86, 248, 323]


In [9]:
# Set the tempo
tempo = mido.bpm2tempo(30)

# Set the ticks_per_beat value
ticks_per_beat = 960*2

# Calculate the duration of a quarter note in ticks
quarter_note_ticks = mido.second2tick(1.0 / 4, ticks_per_beat, tempo)

print(f"Duration of a quarter note in ticks: {quarter_note_ticks}")

Duration of a quarter note in ticks: 240.0


#### Create a MIDI file

In [10]:
# create a new MIDI file
new_midi_file = mido.MidiFile()
new_midi_file.ticks_per_beat = 960

# create a new MIDI track
new_track = mido.MidiTrack()

# set the tempo
tempo = mido.bpm2tempo(30) 
tempo_message = mido.MetaMessage('set_tempo', tempo=tempo)
new_track.append(tempo_message)

for i in range(len(MIDI_rhythms)):
  new_track.append(mido.Message('note_on', note=MIDI_pitches[i], velocity=64, time=MIDI_rhythms[i]))
  new_track.append(mido.Message('note_off', note=MIDI_pitches[i], velocity=64, time=MIDI_rhythms[i])) #TODO: fix
print(MIDI_rhythms)

# add the track
new_midi_file.tracks.append(new_track)

# save the MIDI file
new_midi_file.save('MLP-midi.mid')


[86, 248, 323]


#### Convert the MIDI file into a .wav file (default SR: 44.1kHz)

In [13]:
# convert to .wav
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!fluidsynth -ni font.sf2 MLP-midi.mid -F MLP-wav.wav -r 44100

Audio('MLP-wav.wav')

FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'MLP-wav.wav'..
